In [2]:
!pip install pyspark
import pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 3.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488491 sha256=e985720f3bfd490ccbefe89558dc21bd84c60dafdf3f9123736c30ad3b5a8554
  Stored in directory: /root/.cache/pip/wheels/80/1d/60/2c256ed38dddce2fdd93be545214a63e02fbd8d74fb0b7f3a6
Successfully built pyspark


In [3]:
#Importing necessary libraries
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *

sc=SparkSession.builder.appName("SuperStore").getOrCreate()


In [4]:
# Read the Excel file using pandas
import pandas as pd
excel_file_path = '/content/SampleSuperstore-191212-222546.xls'
pandas_df = pd.read_excel(excel_file_path)

# Convert pandas DataFrame to Spark DataFrame
sample_store = sc.createDataFrame(pandas_df)

# Show the first 5 rows of the Spark DataFrame
sample_store.show(5)

+------+--------------+-------------------+-------------------+--------------+-----------+---------------+---------+--------------+---------------+----------+-----------+------+---------------+---------------+------------+--------------------+------------------+--------+--------+-------------------+
|Row ID|      Order ID|         Order Date|          Ship Date|     Ship Mode|Customer ID|  Customer Name|  Segment|Country/Region|           City|     State|Postal Code|Region|     Product ID|       Category|Sub-Category|        Product Name|             Sales|Quantity|Discount|             Profit|
+------+--------------+-------------------+-------------------+--------------+-----------+---------------+---------+--------------+---------------+----------+-----------+------+---------------+---------------+------------+--------------------+------------------+--------+--------+-------------------+
|     1|CA-2018-152156|2018-11-08 00:00:00|2018-11-11 00:00:00|  Second Class|   CG-12520|    Cla

<h1>Visual assessment<h1/>


1. Checking datatypes
2. Counting number of columns and rows
3. Summary statistics of numerical columns
4. Checking nulls
5. Checking most recent order date
6. Checking distinct types of ship modes, categories, segments, regions and their counts

In [5]:
#To check the datatypes
sample_store.printSchema()

root
 |-- Row ID: long (nullable = true)
 |-- Order ID: string (nullable = true)
 |-- Order Date: timestamp (nullable = true)
 |-- Ship Date: timestamp (nullable = true)
 |-- Ship Mode: string (nullable = true)
 |-- Customer ID: string (nullable = true)
 |-- Customer Name: string (nullable = true)
 |-- Segment: string (nullable = true)
 |-- Country/Region: string (nullable = true)
 |-- City: string (nullable = true)
 |-- State: string (nullable = true)
 |-- Postal Code: double (nullable = true)
 |-- Region: string (nullable = true)
 |-- Product ID: string (nullable = true)
 |-- Category: string (nullable = true)
 |-- Sub-Category: string (nullable = true)
 |-- Product Name: string (nullable = true)
 |-- Sales: double (nullable = true)
 |-- Quantity: long (nullable = true)
 |-- Discount: double (nullable = true)
 |-- Profit: double (nullable = true)



In [6]:
#Counting number of columns and rows
cols = len(sample_store.columns)
rows = sample_store.count()

print(f"Number of columns: {cols}")
print(f"Number of rows: {rows}")

Number of columns: 21
Number of rows: 9994


In [7]:
#To see the summary statistics of numerical columns
numerical_columns = ['Sales','Quantity','Discount','Profit']
sample_store.select(numerical_columns).describe().show()

+-------+-------------------+------------------+-------------------+------------------+
|summary|              Sales|          Quantity|           Discount|            Profit|
+-------+-------------------+------------------+-------------------+------------------+
|  count|               9994|              9994|               9994|              9994|
|   mean| 229.85800083049725| 3.789573744246548|0.15620272163298335|28.656896307784628|
| stddev|   623.245100508681|2.2251096911414017|0.20645196782571595|234.26010769095723|
|    min|0.44399999999999995|                 1|                0.0|-6599.978000000001|
|    max|           22638.48|                14|                0.8| 8399.975999999999|
+-------+-------------------+------------------+-------------------+------------------+



In [8]:
#Checking for nulls
sample_store.filter(col("Order ID").isNull()).show()


+------+--------+----------+---------+---------+-----------+-------------+-------+--------------+----+-----+-----------+------+----------+--------+------------+------------+-----+--------+--------+------+
|Row ID|Order ID|Order Date|Ship Date|Ship Mode|Customer ID|Customer Name|Segment|Country/Region|City|State|Postal Code|Region|Product ID|Category|Sub-Category|Product Name|Sales|Quantity|Discount|Profit|
+------+--------+----------+---------+---------+-----------+-------------+-------+--------------+----+-----+-----------+------+----------+--------+------------+------------+-----+--------+--------+------+
+------+--------+----------+---------+---------+-----------+-------------+-------+--------------+----+-----+-----------+------+----------+--------+------------+------------+-----+--------+--------+------+



In [9]:
#Checking the most recent order date
sample_store.select('Order Date').orderBy(col('Order Date').desc()).show(1)


+-------------------+
|         Order Date|
+-------------------+
|2019-12-30 00:00:00|
+-------------------+
only showing top 1 row



In [11]:
#Checking distinct types of ship modes and their counts
sample_store.groupBy('Ship Mode').count().show()

+--------------+-----+
|     Ship Mode|count|
+--------------+-----+
|   First Class| 1538|
|      Same Day|  543|
|  Second Class| 1945|
|Standard Class| 5968|
+--------------+-----+



In [12]:
#Checking distinct Segments and their counts

sample_store.groupBy('Segment').count().show()

+-----------+-----+
|    Segment|count|
+-----------+-----+
|   Consumer| 5191|
|Home Office| 1783|
|  Corporate| 3020|
+-----------+-----+



In [13]:
#Checking distinct region and their counts
sample_store.groupBy('Region').count().show()

+-------+-----+
| Region|count|
+-------+-----+
|  South| 1620|
|Central| 2323|
|   East| 2848|
|   West| 3203|
+-------+-----+



In [14]:
#Checking distinct category and their counts
sample_store.groupBy('Category').count().show()

+---------------+-----+
|       Category|count|
+---------------+-----+
|Office Supplies| 6026|
|      Furniture| 2121|
|     Technology| 1847|
+---------------+-----+



<h1>Data Exploration<h1/>

1. Total sales by region
2. Top customers by sales
3. Profit margin by product
4. Best selling products(terms of sales)

In [15]:
#Total sales by region
sample_store.select('Region','Sales').groupBy('Region').sum().show()

+-------+-----------------+
| Region|       sum(Sales)|
+-------+-----------------+
|  South|391721.9050000004|
|Central|501239.8907999996|
|   East|678781.2400000002|
|   West|725457.8245000013|
+-------+-----------------+



In [16]:
#Top customers by sales
sample_store.select('Customer Name', 'Sales') \
                             .groupBy('Customer Name') \
                             .agg(sum('Sales').alias('Total Sales')) \
                             .orderBy(desc('Total Sales')) \
                             .show(10)

+------------------+------------------+
|     Customer Name|       Total Sales|
+------------------+------------------+
|       Sean Miller|          25043.05|
|      Tamara Chand|19052.217999999993|
|      Raymond Buch|         15117.339|
|      Tom Ashbrook|          14595.62|
|     Adrian Barton|14473.570999999998|
|      Ken Lonsdale|         14175.229|
|      Sanjit Chand|14142.333999999999|
|      Hunter Lopez|12873.297999999999|
|      Sanjit Engle|12209.438000000002|
|Christopher Conant|12129.071999999998|
+------------------+------------------+
only showing top 10 rows



In [17]:
#Profit margins by product

# Calculating revenue considering discounts (sales - discount)
sample_store =sample_store.withColumn("Revenue", col("Sales") - col("Discount"))

# Calculating cost price per unit (profit / quantity)
sample_store = sample_store.withColumn("Cost_Per_Unit", col("Profit") / col("Quantity"))

# Calculating profit margin percentage (profit / revenue * 100)
sample_store = sample_store.withColumn("Profit_Margin_Percent", (col("Profit") / col("Revenue")) * 100.0)


# Filter top 10 products by Profit
top_profit = sample_store.select('Product Name','Profit','Profit_Margin_Percent').orderBy(col("Profit").desc()).limit(10)

# Show the results
top_profit.show(truncate = False)

+---------------------------------------------------------------------------+------------------+---------------------+
|Product Name                                                               |Profit            |Profit_Margin_Percent|
+---------------------------------------------------------------------------+------------------+---------------------+
|Canon imageCLASS 2200 Advanced Copier                                      |8399.975999999999 |48.0                 |
|Canon imageCLASS 2200 Advanced Copier                                      |6719.980799999999 |48.0                 |
|Canon imageCLASS 2200 Advanced Copier                                      |5039.9856         |48.00000000000001    |
|GBC Ibimaster 500 Manual ProClick Binding System                           |4946.37           |50.0                 |
|Ibico EPK-21 Electric Binding System                                       |4630.4755000000005|49.0                 |
|Canon imageCLASS 2200 Advanced Copier          

In [19]:
#Best selling products
sample_store.select('Product Name','Sales')\
            .groupBy('Product Name')\
            .agg(sum('Sales').alias('Total sales'))\
            .orderBy(desc('Total sales'))\
            .show(10,truncate = False)

+---------------------------------------------------------------------------+------------------+
|Product Name                                                               |Total sales       |
+---------------------------------------------------------------------------+------------------+
|Canon imageCLASS 2200 Advanced Copier                                      |61599.824         |
|Fellowes PB500 Electric Punch Plastic Comb Binding Machine with Manual Bind|27453.384         |
|Cisco TelePresence System EX90 Videoconferencing Unit                      |22638.48          |
|HON 5400 Series Task Chairs for Big and Tall                               |21870.576         |
|GBC DocuBind TL300 Electric Binding System                                 |19823.479         |
|GBC Ibimaster 500 Manual ProClick Binding System                           |19024.5           |
|Hewlett Packard LaserJet 3310 Copier                                       |18839.686         |
|HP Designjet T520 Inkjet Larg